In [1]:
import pandas as pd
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

recalls = pd.read_csv("Recall data/All_recall_data")
recalls = recalls.drop(['Unnamed: 0'], axis = 1)

In [2]:
unsorted_recall_data = Counter(recalls["FIRM_NAME"])
recall_data = unsorted_recall_data.most_common()

lst = []
for i in list(range(len(recall_data))):
    if recall_data[i][1] >= 10:
        lst.append(recall_data[i])

It is not accurate to assume that just because a medical firm recalls a lot of devices, that it is a bad firm. It may simply be larger than other firms, thus producing more devices. To rectify this, I assign each of the firms we are considering (the firms with over 10 recalls) a "recall score", where $$ \text{recall score} = \frac{\text{total # days before recall addressed}}{\text{# recalls}}. $$ This will "normalize" the data, and give us one measure of how reliable a firm is. That is, how quickly we can expect them to address a recalled product.

The immediate issue is that, out of the firms with over 10 products recalled in the 2010-2014 timespan ("lst"), some have similar names. We group the similarly named firms into umbrella firms by the following (considerable) hard-code.

In [3]:
main_firms = {
    "Stryker": ["Stryker Endoscopy", "Stryker Howmedica Osteonics Corp.", "Stryker Instruments Div. of Stryker Corporation", 'Stryker Craniomaxillofacial Division','Stryker Instruments Division of Stryker Corporation', 'Stryker Medical Division of Stryker Corporation', 'Stryker Spine', 'Stryker Neurovascular', 'Stryker Neurovascular', 'Stryker Communications Corp', 'Orthovita, Inc., dBA Stryker Orthobiologics.', 'Stryker Medical Div. of Stryker Corporation' ], 
    "Customed": ["Customed, Inc"], 
    "Zimmer" : ["Zimmer, Inc.", 'Zimmer Inc.'],
    'GE Healthcare' : ['GE Healthcare, LLC', 'GE OEC Medical Systems, Inc', 'Ge Healthcare It', 'GE Healthcare', 'GE Healthcare It', 'Ge Healthcare, Llc'],
    'Terumo Cardiovascular Systems' : ['Terumo Cardiovascular Corporation', 'Terumo Cardiovascular Systems Corp', 'Terumo Cardiovascular Systems Corporation'],
    'Boston Scientific' : ['Boston Scientific Corporation', 'Boston Scientific CRM Corp', 'Boston Scientific Corporation, Neurovascular, 47900 Bayside Parkway, Fremont, CA 94538'],
    'Biomet' : ['Biomet, Inc.', 'Biomet Microfixation, LLC', 'Biomet 3i, LLC', 'Biomet Microfixation, Inc.'],
    'Beckman' : ['Beckman Coulter Inc.'],
    'Hospira' : ['Hospira Inc.'],
    'Cook' : ['Cook, Inc.'],
    'Biomerieux' : ['Biomerieux Inc'],
    'The Anspach Effort' : ['The Anspach Effort, Inc.'],
    'Smith & Nephew' : ['Smith & Nephew Inc', 'Smith & Nephew, Inc. Endoscopy Division', 'Smith & Nephew Inc.'],
    'Siemens' : ['Siemens Medical Solutions USA, Inc - Radiation Oncology', 'Siemens Healthcare Diagnostics Inc', 'Siemens Healthcare Diagnostics','Siemens Healthcare Diagnostics, Inc.', 'Siemens Medical Solutions USA, Inc', 'Siemens Medical Solutions USA,  Inc', 'Siemens Healthcare Diagnostics, Inc', 'Siemens Medical Solutions USA, Inc.' ],
    "Philips" : ['Philips Healthcare Inc.', 'Philips Medical Systems (Cleveland) Inc', 'Philips Medical Systems North America Co.   Phillips', 'Philips Medical Systems', 'Philips Medical Systems, Inc.'],
    'ICU Medical' : ['ICU Medical, Inc.'],
    'Baxter Healthcare' : ['Baxter Healthcare Corp.'],
    'Teleflex Medical' : ['Teleflex Medical', 'Arrow International, Inc., Division of Teleflex Medical Inc.','Arrow International Inc'],
    'Heritage Labs' : ['Heritage Labs International LLC'],
    'Depuy' : ['DePuy Spine, Inc.', 'Depuy Orthopaedics, Inc.', 'DePuy Orthopaedics, Inc.', 'DePuy Mitek, Inc., a Johnson & Johnson Co.'],
    'Ortho-Clinical Diagnostics' : ['Ortho-Clinical Diagnostics'],
    'Centurion' : ['Centurion Medical Products Corporation', 'Centurion Medical Products'],
    'Angiodynamics' : ['Angiodynamics Worldwide Headquarters', 'Angiodynamics, Inc.', 'Angiodynamics, INC'],
    'Smiths Medical' : ['Smiths Medical ASD, Inc.', 'Smiths Medical'],
    'Integra LifeSciences' : ['Integra LifeSciences Corp.', 'Integra LifeSciences Corporation', 'Integra LifeSciences Corp. d.b.a. Integra Pain Management'],
    'Intuitive Surgical' : ['Intuitive Surgical, Inc.'],
    'Sorin Group' : ['Sorin Group USA, Inc.', 'Sorin Group Deutschland GmbH'],
    'Codman & Shurtleff' : ['Codman & Shurtleff, Inc.'],
    'LumiQuick Diagnostics' : ['LumiQuick Diagnostics Inc.'],
    'Elekta' : ['Elekta, Inc.'],
    'Becton Dickinson' : ['Becton Dickinson & Co.', 'Becton Dickinson Medical Systems', 'Becton Dickinson & Company'],
    'Covidien' : ['Covidien LP', 'Nellcor Puritan Bennett Inc. (dba Covidien Ltd)', 'Covidien LLC','Nellcor Puritan Bennett Inc. (dba Covidien LP)'],
    'Synthes' : ['Synthes USA (HQ), Inc.', 'Synthes USA HQ, Inc.', 'Synthes, Inc.'],
    'Multi-Med' : ['Multi-Med, Inc.'],
    'Varian Medical Systems' : ['Varian Medical Systems, Inc. Oncology Systems', 'Varian Medical Systems, Inc.', 'Varian Medical Systems Oncology Systems'],
    'Nurse Assist' : ['Nurse Assist, Inc'],
    'Merit Medical' : ['Merit Medical Systems, Inc.'],
    'Instrumed International' : ['Instrumed International, Inc.'],
    'Carestream Health' : ['Carestream Health Inc.','Carestream Health, Inc.'],
    'Medtronic' : ['Medtronic Navigation, Inc.', 'Medtronic Inc. Cardiac Rhythm Disease Management', 'Medtronic Sofamor Danek USA Inc', 'Medtronic Neuromodulation', 'Medtronic Inc. Cardiac Rhythm Disease Managment', 'Medtronic Vascular'],
    'Toshiba' : ['Toshiba American Medical Systems Inc'],
    'ConMed' : ['ConMed Corporation', 'Linvatec Corp. dba ConMed Linvatec', 'ConMed Electrosurgery'],
    'Roche' : ['Roche Diagnostics Operations, Inc.', 'Roche Molecular Systems, Inc.', 'Roche Insulin Delivery Systems Inc.'],
    'Fresenius Medical Care Holdings' : ['Fresenius Medical Care Holdings, Inc.'],
    'King Systems' : ['King Systems Corp.'],
    'Zevex International' : ['Zevex International, Inc.'],
    'Medical Device Technologies' : ['Medical Device Technologies, Inc.'],
    'Abbott' : ['Abbott Laboratories', 'Abbott Diabetes Care, Inc.', 'Abbott Point Of Care Inc.', 'Abbott Point of Care Inc.', 'Abbott Molecular'],
    'Wright Medical Technology' : ['Wright Medical Technology Inc'],
    'Innovatech Medical Resources' : ['Innovatech Medical Resources L.P.'],
    'Instrumentation Laboratory' : ['Instrumentation Laboratory Co.'],
    'Atrium Medical' : ['Atrium Medical Corporation'],
    'Ethicon' : ['Ethicon Endo-Surgery Inc', 'Ethicon, Inc.'],
    'Advanced Sterilization' : ['Advanced Sterilization Products'],
    'Applied Medical Resources' : ['Applied Medical Resources Corp'],
    'Hill-Rom' : ['Hill-Rom, Inc.'],
    'Bausch & Lomb' : ['Bausch & Lomb Inc'],
    'Remel' : ['Remel Inc'],
    'MOOG' : ['MOOG Medical Devices Group'],
    'SpineFrontier' : ['SpineFrontier, Inc.'],
    'Steris' : ['Steris Corporation'],
    'Argon Medical Devices' : ['Argon Medical Devices, Inc'],
    'Endotec' : ['Endotec, Inc.'],
    'Edwards Lifesciences' : ['Edwards Lifesciences, LLC'],
    'Computerized Medical Systems' : ['Computerized Medical Systems Inc'],
    'Cardinal' : ['Cardinal Health', 'Cardinal Health, Medical Products & Services'],
    'NuVasive' : ['NuVasive Inc'],
    'AGFA' : ['AGFA Corp.'],
    'Navilyst Medical' : ['Navilyst Medical, Inc'],
    'SCC' : ['SCC Soft Computer'],
    'DeRoyal Industries' : ['DeRoyal Industries Inc', 'Deroyal Industries, Inc. Lafollette'],
    'Baxa' : ['Baxa Corporation'],
    'Cardiac Science' : ['Cardiac Science Corporation'],
    'Oridion Medical' : ['Oridion Medical 1987 Ltd.'],
    'Leica Microsystems' : ['Leica Microsystems, Inc.'],
    'Orthopediatrics' : ['Orthopediatrics Corp', 'OrthoPediatrics Corp'],
    'Aspen Surgical Products' : ['Aspen Surgical Products, Inc.'],
    'Spacelabs Healthcare' : ['Spacelabs Healthcare, Llc'],
    'Churchill Medical Systems' : ['Churchill Medical Systems, Inc.'],
    'Bio-Rad Laboratories' : ['Bio-Rad Laboratories Inc', 'Bio-Rad Laboratories, Inc.'],
    'Alere' : ['Alere San Diego, Inc.', 'Alere San Diego'],
    'Calbiotech' : ['Calbiotech Inc'],
    'C.R. Bard' : ['Bard Peripheral Vascular Inc', 'Bard Access Systems', 'Davol, Inc., Subs. C. R. Bard, Inc.', 'Davol, Inc., Sub. C. R. Bard, Inc.'],
    'Tripath Imaging' : ['Tripath Imaging, Inc.'],
    'Invacare' : ['Invacare Corporation'],
    'Exactech' : ['Exactech, Inc.'],
    'Maquet Cardiovascular' : ['Maquet Inc.', 'Maquet Cardiovascular, LLC', 'Maquet Cardiovascular Us Sales, Llc'],
    'Encore Medical' : ['Encore Medical, Lp'],
    'Stanley Security Solutions' : ['Stanley Security Solutions, Inc.'],
    'Biosense Webster' : ['Biosense Webster, Inc.'],
    'Surgical Instrument Service And Savings' : ['Surgical Instrument Service And Savings, Inc.'],
    'Cordis' : ['Cordis Corporation'],
    'Church & Dwight' : ['Church & Dwight Inc'],
    'Spinal Elements' : ['Spinal Elements, Inc'],
    'Waldemar Link GmbH & Co. KG' : ['Waldemar Link GmbH & Co. KG (Corp. Hq.)'],
    'Encision' : ['Encision, Inc.'],
    'Vygon' : ['Vygon Corporation'],
    'Progressive Medical' : ['Progressive Medical Inc'],
    'Kimberly-Clark' : ['Kimberly-Clark Corporation'],
    'Radiometer America' : ['Radiometer America Inc'],
    'Ortho Development' : ['Ortho Development Corporation'],
    'Circaid Medical Products' : ['Circaid Medical Products Inc'],
    'Gambro Renal Products' : ['Gambro Renal Products, Incorporated'],
    'LeMaitre Vascular' : ['LeMaitre Vascular, Inc.'],
    'Medrad' : ['Medrad Inc'],
    'Pointe Scientific' : ['Pointe Scientific, Inc.'],
    'BioCheck' : ['BioCheck Inc'],
    'Sanofi' : ['Genzyme Corporation, A Sanofi Company'],
    'AMO' : ['AMO Puerto Rico Manufacturing, Inc.', 'AMO Manufacturing USA, LLC'],
    'Winco Mfg.' : ['Winco Mfg., LLC'],
    'Custom Medical Specialties' : ['Custom Medical Specialties, Inc.'],
    'Karl Storz Endoscopy America' : ['Karl Storz Endoscopy America Inc'],
    'Cincinnati Sub-Zero Products' : ['Cincinnati Sub-Zero Products Inc'],
    'Westmed' : ['Westmed Inc'],
    'Osteotech' : ['Osteotech Inc'],
    'Trumpf Medical Systems' : ['Trumpf Medical Systems, Inc.'],
    'CareFusion 303' : ['CareFusion 303, Inc.', 'Carefusion 2200 Inc'],
    'Ascent' : ['Ascent Healthcare Solutions'],
    'Civco Medical Instruments' : ['Civco Medical Instruments Inc'],
    'Nova Biomedical' : ['Nova Biomedical Corporation'],
    'Aesculap' : ['Aesculap, Inc.'],
    'Convatec' : ['Convatec Inc.'],
    'Ansell' : ['Ansell Healthcare Products LLC', 'Ansell Sandel Medical Industries LLC'],
    'Horiba Instruments' : ['Horiba Instruments, Inc dba Horiba Medical'],
    'Olympus America' : ['Olympus America Inc.'],
    'Orthohelix Surgical Designs' : ['Orthohelix Surgical Designs Inc'],
    'Cooper Surgical' : ['Cooper Surgical, Inc.'],
    'Animas' : ['Animas Corporation'],
    'Penner' : ['Penner Mfg Inc'],
    'B. Braun Medical' : ['B. Braun Medical, Inc.'],
    'Superstat' : ['Superstat Corp'],
    'Accuray' : ['Accuray Incorporated'],
    'Draeger Medical Systems' : ['Draeger Medical Systems, Inc.'],
    'Physio Control' : ['Physio Control, Inc.'],
    'Advanced Orthopaedic Solutions' : ['Advanced Orthopaedic Solutions Inc'],
    'Kerr' : ['Kerr Corporation'],
    'Unimed Surgical Products' : ['Unimed Surgical Products, Inc.'],
    'Ventlab' : ['Ventlab Corporation'],
    'Symbios Medical Products' : ['Symbios Medical Products, LLC'],
    'Nitinol Devices and Components' : ['Nitinol Devices and Components, Inc.'],
    'Medical Action Industries' : ['Medical Action Industries Inc'],
    'Hologic' : ['Hologic, Inc.'],
    'Sybron' : ['Sybron Dental Specialties'],
    'Mallinckrodt' : ['Mallinckrodt Inc'],
    'Alcon Research' : ['Alcon Research LTD dba Alcon Laboratories, Inc.'],
    'Teco' : ['Teco Diagnostics'],
    'St Jude Medical' : ['St Jude Medical CRMD'],
    'Interventional Spine' : ['Interventional Spine Inc'],
    'Raysearch' : ['RAYSEARCH LABORATORIES AB'],
    'NxStage Medical' : ['NxStage Medical, Inc.'],
    'Spinal Solutions' : ['Spinal Solutions, LLC'],
    'Medline Industries' : ['Medline Industries, Inc.'],
    'Nanosphere' : ['Nanosphere, Inc.'],
    'Lab Vision' : ['Lab Vision Corporation'],
    'Steritec Products' : ['Steritec Products Mfc. Co., Inc.'],
    '3M' : ['3M Company / Medical Division'],
    'BD Biosciences' : ['BD Biosciences, Systems & Reagents'],
    'Vital Images' : ['Vital Images, Inc.'],
    'Alphatec Spine' : ['Alphatec Spine, Inc.'],
    'American Medical Systems' : ['American Medical Systems, Inc.'],
    'Arjo' : ['Arjo, Inc. dba ArjoHuntleigh'],
    'Del Mar Reynolds Medical' : ['Del Mar Reynolds Medical, Ltd.'],
    'Mindray' : ['Mindray DS USA, Inc. d.b.a. Mindray North America'],
    'Orthopedic Alliance' : ['Orthopedic Alliance LLC'],
    'ASO' : ['ASO, LLC']
}

In [4]:
keys_list = list(main_firms.keys())
main_firms_val = {x:0 for x in keys_list}
total_recalled = {x:0 for x in keys_list}
recall_scores = {x:0 for x in keys_list}

So, listed under "main_firms" are the names of the firms we are considering, and listed under "main_firms_val" is a (currently empty) score correponding to each main firm. We now sum the total amount of days that a product was recalled and left unaddressed per firm. That is,
$$ \text{main firm val(firm_i)} = \sum_{j = 1}^{n}(\text{'TERMINATION_DT'})-(\text{'POSTED_INTERNET_DT'}), $$
for $1 \leq n \leq \text{'total_recalled'}$ for all main firms $1 \leq i \leq \text{recall_interesting ($167$)}$.

In [5]:
for i in main_firms.items():
    for j in list(range(len(i[1]))):
        for k in list(range(len(lst))):
            if lst[k][0] in i[1]:
                total_recalled[str(i[0])] += lst[k][1]
                
lst_of_lst = []
for i in list(range(len(lst))):
    lst_of_lst.append(lst[i][0])
    
columns = list(recalls)
recalls_interesting = pd.DataFrame(columns=columns)

#this will "clean" the dataframe of uninteresting firms
#not actually sure if it worked...
for i in list(range(len(recalls))):
    if recalls["FIRM_NAME"].iloc[i] in lst_of_lst:
        recalls_interesting = recalls_interesting.append(recalls.iloc[i], ignore_index=True)

The following cell will determine the total amount of days each firm has had a recalled product un-addressed. If that firm chose to not address the issue (that is, "TERMINATION_DT == NaN"), they were punished by getting an additional score of "avg" for each unaddressed product.

In [6]:
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y/%m/%d")
    d2 = datetime.strptime(d2, "%Y/%m/%d")
    return abs((d2 - d1).days)

number = 0
for i in list(range(len(recalls_interesting))):
    try:
        number += days_between( ((str(recalls_interesting['POSTED_INTERNET_DT'].iloc[i])).split(" "))[0], ((str(recalls_interesting["TERMINATION_DT"].iloc[i]).split(" "))[0]))
    except:
        number += 0
                
avg = int(number/len(recalls_interesting))

for i in list(range(len(recalls_interesting))):
    for j in main_firms.items():
        if recalls_interesting["FIRM_NAME"].iloc[i] in j[1]:
            try:
                main_firms_val[j[0]] += days_between( ((str(recalls_interesting['POSTED_INTERNET_DT'].iloc[i])).split(" "))[0], ((str(recalls_interesting["TERMINATION_DT"].iloc[i]).split(" "))[0]))
            except:
                main_firms_val[j[0]] += avg
    

In [7]:
for i in main_firms.items():
    recall_scores[i[0]] += int(main_firms_val[i[0]] / total_recalled[i[0]])

We now have a measure of how reliable each firm is! The lower a firm's 'recall_score' is, the more reliable it is; and the higher a firm's 'recall score' is, the less reliable it is. If a firm has a 'recall_score' of 0, then it has addressed every recalled product the day it was recalled.

In [8]:
print('RECALL SCORES:')
print()
for i in recall_scores.items():
    print(str(i[0]) + ": " + str(i[1]))

RECALL SCORES:

Stryker: 29
Customed: 1038
Zimmer: 262
GE Healthcare: 88
Terumo Cardiovascular Systems: 131
Boston Scientific: 167
Biomet: 89
Beckman: 470
Hospira: 977
Cook: 142
Biomerieux: 516
The Anspach Effort: 310
Smith & Nephew: 263
Siemens: 61
Philips: 171
ICU Medical: 90
Baxter Healthcare: 570
Teleflex Medical: 266
Heritage Labs: 348
Depuy: 162
Ortho-Clinical Diagnostics: 1338
Centurion: 44
Angiodynamics: 410
Smiths Medical: 428
Integra LifeSciences: 116
Intuitive Surgical: 165
Sorin Group: 89
Codman & Shurtleff: 756
LumiQuick Diagnostics: 32
Elekta: 486
Becton Dickinson: 67
Covidien: 216
Synthes: 353
Multi-Med: 899
Varian Medical Systems: 148
Nurse Assist: 419
Merit Medical: 212
Instrumed International: 1090
Carestream Health: 394
Medtronic: 63
Toshiba: 523
ConMed: 196
Roche: 133
Fresenius Medical Care Holdings: 777
King Systems: 484
Zevex International: 237
Medical Device Technologies: 66
Abbott: 87
Wright Medical Technology: 352
Innovatech Medical Resources: 260
Instrumentati

In [9]:
#Find a way to visualize
#This is a work in progress



# # lists = sorted(recall_scores.items()) # sorted by key, return a list of tuples

# # x, y = zip(*lists) # unpack a list of pairs into two tuples
# # figure(num=None, figsize=(18, 8), dpi=80, facecolor='w', edgecolor='k')
# # plt.xticks(fontsize=8, rotation=90)
# # plt.ylabel("Recall Score")
# # plt.title("Recall Score per company")
# # plt.plot(x, y)
# # # plt.plot(figsize=(50,10))
# # plt.show()

# names = list(recall_scores.keys())
# values = list(recall_scores.values())

# #tick_label does the some work as plt.xticks()
# figure(num=None, figsize=(18, 8), dpi=80, facecolor='w', edgecolor='k')

# plt.bar(range(len(recall_scores)),values,tick_label=names)
# plt.xticks(fontsize=8, rotation=90)
# # plt.savefig('bar.png')

# plt.show()